### GitHub и система контроля версий

1. Сделать форк от этой репки
2. Сделать тестовое
3. Создать мерж ревест с твоего форка на мою репку.

Принцип тестового следующий - 
Работаем в пайчарме, пишем в питоне или тетрадке, на выбор</p> Результат выполненой работы - тестовые данные выгружены из json и положены в поднятую базу,
к ним написаны sql запросы, результат каждого запроса распечатан.

***
**FORK** - это копия репозитория (branch в Git) </p> Вилка — это новый репозиторий, который совместно использует параметры кода и видимости с исходным "вышестоящим" репозиторием.<p> Программное обеспечение с открытым кодом строится на той идее, что, предоставляя общий доступ к коду, мы можем создавать более надежное и эффективное ПО.

pull request - запрос на слияние репозитория измененного и оригинального репозитория

***
# Тестовое задание:

На основании данных необходимо считать следующий набор метрик:

* Количество показов и уникальных пользователей за день в разрезе по платформам, в том числе по всем платформам суммарно;
* Количество за день уникальных авторов и уникального контента, показанного в ленте;
* Количество сессий, средняя глубина просмотра (по позиции фида) и средняя продолжительность пользовательской сессии в ленте за день.

Напишите алгоритмы / запросы для их расчета.

Если это необходимо – допускается создание промежуточных объектов.

### Описание данных
Данные по показам в ленте. Одна строчка – показ одного «фида» в ленте.

Фидом называем один логический элемент
Описание полей:

* durationMs – время в течение которого пользователь видел фид
* position – позиция в ленте на которой был показан фид
* owners – создатели контента в фиде
* resources – ссылки на ID контента, содержащегося в фиде






json - это формат обмена данными, которые можно передать с клиента на сервер и с сервера на клиент. Используется для связи между приложениями. Используется при API rest.

In [1]:
import json
import pandas as pd
import datetime
import numpy as np
from typing import List, Union
from datetime import datetime as dt


In [2]:
file = 'feeds_show.json'

In [3]:
# read json file into DataFrame:
data = [json.loads(line) for line in open(file, 'r')]
df = pd.DataFrame(data)

In [4]:
#convert timestamp to date:
df['timestamp'] = df['timestamp']\
                        .apply(lambda x: datetime.datetime.fromtimestamp(int(x)/1000).strftime("%Y-%m-%d %H:%M:%S"))

df['timestamp'] = pd.to_datetime(df.timestamp,format='%Y-%m-%d %H:%M:%S')

df['Date'] = df['timestamp'].dt.date

In [5]:
columns_df = ['userId', 'platform', 'durationMs', 'position', 'Date', 'timestamp', 'owners','resources' ]
df = df.reindex(columns=columns_df)

In [6]:
columns = ['userId', 'platform', 'durationMs', 'position', 'Date', 'timestamp', '_owners', 'owners_values', 'resources']

#function convert json to pandas dataframe #1 with users
def get_df_users(df: pd.DataFrame, columns: List[str] ) -> pd.DataFrame:
    for index, row in df.iterrows():
        for elem in row['owners']:

            key   = elem
            values = row['owners'][elem]
            for value in values:

                df.at[index,'_owners']=key    
                df.at[index,'owners_values']=value        

        
    return df[columns] 
    

In [7]:
start = dt.now()
df_users = get_df_users(df, columns)
total_duration = dt.now() - start
print(f"Time of creating Table 'users' is {total_duration}") 

Time of creating Table 'users' is 0:00:05.925363


In [8]:
df_users_sql = df_users.iloc[:,:8]


In [9]:
df_resources = df_users[['Date','timestamp', '_owners','owners_values', 'resources']]
columns_resources = ['Date', 'timestamp', '_owners','owners_values', 'resources', 'resources_id' ]
#df_input = pd.DataFrame(columns=columns_resources)
list_input=[]

#function convert json to pandas dataframe #2 with resources
def get_df_resources(df: pd.DataFrame ) -> pd.DataFrame:
    for index, row in df_resources.iterrows():
        for elem in row['resources']:

            key   = elem
            values = row['resources'][elem]
            for value in values:
                new_row = {'Date': row['Date'],
                           'timestamp': row['timestamp'],
                           '_owners': row['_owners'],
                           'owners_values': row['owners_values'],
                           'resources': key,
                           'resources_id': value}
                list_input.append(new_row)

                #df_input.loc[len(df_input.index)] = new_row 
    return pd.DataFrame(list_input, columns = columns_resources)
            



In [10]:
start = dt.now()
df_resources_sql = get_df_resources(df_resources)
total_duration = dt.now() - start
print(f"Time of creating Table 'resources' is {total_duration}") 

Time of creating Table 'resources' is 0:00:05.373226


#### Connecting to PostgreSQL Database using SQLAlchemy in Python

In [11]:
import psycopg2
import psycopg2.extras
from bs4 import BeautifulSoup
import sys
import warnings
if not sys.warnoptions:
       warnings.simplefilter("ignore")

from sqlalchemy import create_engine

In [12]:
#### Connecting to PostgreSQL Database using SQLAlchemy in Python

# DEFINE THE DATABASE CREDENTIALS
user = 'postgres'
password = '123456'
host = 'localhost'
port = 5432
database = 'db_test'

In [13]:
# CONNECTION TO DB
try:
    conn = psycopg2.connect (
        host = host,
        dbname = database,
        user = user,
        password = password,
        port = port)
    #conn.close()
except Exception as error:
    print(error)

In [14]:
cur = conn.cursor(cursor_factory = psycopg2.extras.DictCursor)


In [15]:
# PYTHON FUNCTION TO CONNECT TO THE POSTGRESQL DATABASE AND
# RETURN THE SQLACHEMY ENGINE OBJECT
def get_connection():
    return create_engine(
        url="postgresql://{0}:{1}@{2}:{3}/{4}".format(
            user, password, host, port, database
        )
    )
 

if __name__ == '__main__':
 
    try:
        # GET THE CONNECTION OBJECT (ENGINE) FOR THE DATABASE
        engine = get_connection()
        print(
            f"Connection to the {host} for user {user} created successfully.")
    except Exception as ex:
        print("Connection could not be made due to the following error: \n", ex)



Connection to the localhost for user postgres created successfully.


In [16]:
# download postgresql table
data = df_users_sql
data.to_sql('df_users_sql', engine, if_exists='replace')

318

In [17]:
# download postgresql table
data = df_resources_sql
data.to_sql('df_resources_sql', engine, if_exists='replace')

968

***
## Quering Data in PostgreSQL

#### Количество показов и уникальных пользователей за день в разрезе по платформам, в том числе по всем платформам суммарно

In [18]:
query_1 = """


with CTE as 
(
	SELECT 
	a."Date" as date,
	a.platform as platform,
	count(a.index) as views_,
	count(distinct a."userId")  as unique_users

	FROM public.df_users_sql a
	group by a."Date", a.platform
	order by 1,2 
)
select 
	date, 
	platform,
	views_,
	sum(views_) over (partition by date) as views_by_day,
	unique_users,
	sum(unique_users) over (partition by date) as unique_users_by_day
from CTE
"""

In [19]:
pd.read_sql_query(query_1, conn)

,date,platform,views_,views_by_day,unique_users,unique_users_by_day
0,2019-09-12,APP_ANDROID,18078,31230.0,109,200.0
1,2019-09-12,APP_IOS,2378,31230.0,23,200.0
2,2019-09-12,APP_WINPHONE,247,31230.0,1,200.0
3,2019-09-12,DESKTOP_WEB,9177,31230.0,52,200.0
4,2019-09-12,MOBILE_WEB,1350,31230.0,15,200.0
5,2019-09-13,APP_ANDROID,1793,3088.0,30,47.0
6,2019-09-13,APP_IOS,92,3088.0,4,47.0
7,2019-09-13,DESKTOP_WEB,1175,3088.0,11,47.0
8,2019-09-13,MOBILE_WEB,28,3088.0,2,47.0


#### Количество за день уникальных авторов и уникального контента, показанного в ленте

In [22]:
query_2 = """


with CTE as
(
SELECT t1."Date", t1."_owners", t1."owners_values", t2."resources", t2."resources_id" 
FROM public.df_users_sql t1 LEFT JOIN public.df_resources_sql t2 
ON t1."owners_values" = t2."owners_values" and t1."timestamp" = t2."timestamp"
)
		select 
			a."Date",
			count(distinct a."owners_values") as unique_owners,
			count(distinct a."resources_id") as unique_content
		from CTE a
			group by a."Date"
            
"""

In [23]:
pd.read_sql_query(query_2, conn)

,Date,unique_owners,unique_content
0,2019-09-12,10274,38905
1,2019-09-13,1264,5526


#### Количество сессий, средняя глубина просмотра (по позиции фида) и средняя продолжительность пользовательской сессии в ленте за день.

Глубина просмотра (или посещения) сайта — это среднее число просмотров контента сайта пользователями за одно посещение

In [26]:
query_3 = """

SELECT 
	a."Date",
	a."position",
	count(a.index) as count_session,
	count(a.index)/count( distinct a."userId") as avg_sesion_by_user_sec,
    round(sum(a."durationMs")/count( distinct a."userId")/1000) as avg_sesion_by_user_sec

FROM public.df_users_sql a
group by a."Date", a."position"
order by a."Date", a."position"


"""

In [28]:
pd.read_sql_query(query_3, conn).head(10)

,Date,position,count_session,avg_sesion_by_user_sec,avg_sesion_by_user_sec
0,2019-09-12,1.0,859,4,521.0
1,2019-09-12,2.0,658,4,323.0
2,2019-09-12,3.0,580,3,116.0
3,2019-09-12,4.0,553,3,331.0
4,2019-09-12,5.0,509,3,43.0
5,2019-09-12,6.0,489,3,95.0
6,2019-09-12,7.0,482,3,43.0
7,2019-09-12,8.0,477,3,50.0
8,2019-09-12,9.0,496,3,36.0
9,2019-09-12,10.0,526,3,501.0
